### PROJECT ABSTRACT 

With all the code computed on other notebookes:

 **1- Spotify API** - where I did my first connection to the API, providing it with like and disliked songs, as well as my musical journey data.

 **2- EDA** - where I explored all Spotify features based on the two groups in study.

 **3- My Musical Journey** - also a kind of EDA notebook. However, in this one I analysed my top year playlists to evaluate the evolution of my music taste from 2016 to 2019. 

 **4- Supervised ML** - tested multiple suprevised learning classifiers getting to a 86% Accuracy with Random Forest. I have now a pretty good algorithm that aimns to detect if I like a song or not, as well as the prob of that event happening.

 **5- Test on other's playlists** - used the best algorithm evaluated on ML notebook and applied to outside data - songs from friend's playlists - meaning neither data from the traning nor the test set.

 **6- Unsupervised Learning** - without any song labeling, used the whole dataset (liked + disliked songs) to extract knowledge based on any patterns: Clustering, Dimensionality Reduction.

Finally, in this script I aim to produce a final product where the user can input his/hers/theirs **(public) playlsit URI** and get an  **Exploratory Data Analysis on the songs' features, as well as new song recomendations.** This will be produced based on Unsupervised Learning techniques, since we can assume the shared playlist will be a collection of liked songs and therefore we can't apply labelling techniques.

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import json
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sb
%matplotlib inline
import plotly.express as px

import plotly.figure_factory as ff

from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.transform import Rotation as R
import scipy.cluster.hierarchy as shc

import statsmodels.api as sm
from scipy import stats
from scipy.stats import ttest_1samp, ttest_rel, ttest_ind
from IPython.display import display, Image, SVG, Math, YouTubeVideo


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
username = 'britopereiraa'
client_id ='17a3ed14ab434b33b28f19f8bea3337d'
client_secret = '7f848f9d2a024b15be5fa776b4dc7a05'
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played'

token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
def master_function(username, uri):
    uri = playlist_uri    
    username = username
    playlist_id = uri
    results = {'items':[]}

    for n in range(0,3000,100):
        new = sp.user_playlist_tracks(username, playlist_id,  offset = n)
        results['items'] += new['items']

        playlist_tracks_data = results
        playlist_tracks_id = []
        playlist_tracks_titles = []
        playlist_tracks_artists = []

        for track in playlist_tracks_data['items']:
            playlist_tracks_id.append(track['track']['id'])
            playlist_tracks_titles.append(track['track']['name'])

        #adds a list of all artists involved in the song to the list of artists for the playlist
            for artist in track['track']['artists']:
                artist_list = []
                artist_list.append(artist['name'])
            playlist_tracks_artists.append(artist_list[0])
    
    #some tracks might have none id
    playlist_tracks_id = list(filter(None, playlist_tracks_id)) 
    
    df = pd.DataFrame([])
    for i in range(0, len(playlist_tracks_id)):
        features = sp.audio_features(playlist_tracks_id[i])
        features_df = pd.DataFrame(features)
        df = df.append(features_df)
       

    df['title'] = playlist_tracks_titles
    #features_df['first_artist'] = playlist_tracks_first_artists
    df['main_artist'] = playlist_tracks_artists
    #features_df = features_df.set_index('id')
    df = df[['id', 'title', 'main_artist',
                               'danceability', 'energy', 'key', 'loudness',
                               'mode', 'acousticness', 'instrumentalness',
                               'liveness', 'valence', 'tempo',
                               'duration_ms', 'time_signature']]

    return df

### USER PLAYLIST

In [4]:
#input variables
playlist_uri ="2ERd3YQjKnGSEyInrwYPRh"
username = "João Fanha"

In [5]:
user_df = master_function(username,playlist_uri)
user_df.shape

(57, 15)

### Get the all songs playlist: 

A playlist with over 5000 songs of all Genre 

In [6]:
features = ['danceability', 'energy', 'key','loudness', 'mode', 'acousticness', 'instrumentalness', 'liveness','valence', 'tempo']

#import dataset created in other jupyter notebook
all_songs = pd.read_csv('../data/all_songs.csv')

#drop duplicates
all_songs.drop_duplicates(subset ="id", 
                     keep = False, inplace = True)


#check recomended songs that are not in the playlist already:

all_songs = all_songs.loc[~((all_songs.id.isin(df['id']))),:]
all_songs.head()

NameError: name 'df' is not defined

In [ ]:
all_songs.shape

## Songs Recomendations

### Clustering

In [ ]:
#features scaling 
scaler = StandardScaler()
scaler.fit(all_songs[features])

all_songs_features_scaled = scaler.transform(all_songs[features])

In [ ]:
#TRAIN CLUSTERING IN ALL SONGS

X = all_songs_features_scaled
# Instantiate the clustering model and visualizer
model = KMeans()
visualizer = KElbowVisualizer(model, k=(1,12))

visualizer.fit(X)        # Fit the data to the visualizer
visualizer.show()        # Finalize and render the figure


In [ ]:
#label the data into clusters

kmean = KMeans(n_clusters=4)
kmean.fit(all_songs_features_scaled)

pred = kmean.labels_
all_songs['labels'] = pred

#check clusters
all_songs.head()

In [ ]:
all_songs['labels'].value_counts()

In [ ]:
#APPLY CLUSTERS

df = user_df.copy()
pred_df = kmean.predict(df[features])
df['songs group'] = pred_df

df = df.reset_index()
df = df.drop(columns=['index'])
df.head()

### PCA

In [ ]:
#PCA on all songs

from sklearn.decomposition import PCA

pca = PCA(n_components=3, random_state=42)
df_pca_all_songs = pd.DataFrame(data=pca.fit_transform(all_songs_features_scaled), columns=['PC1','PC2','PC3'])

In [ ]:
pca.components_
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

In [ ]:
plt.matshow(pca.components_, cmap='viridis')
plt.yticks([0, 1, 2], ["First component", "Second component", "Third component"])
plt.colorbar()
plt.xticks(range(len(all_songs[features].columns)),all_songs[features], rotation=60, ha='left')
plt.xlabel("Feature")
plt.ylabel("Principal components")

In [ ]:
df_pca_all_songs = df_pca_all_songs.merge(all_songs, left_index=True, right_index=True)
df_pca_all_songs = df_pca_all_songs[['PC1', 'PC2', 'PC3', 'title', 'main_artist']]

df_pca_all_songs.shape

In [ ]:
#PCA with friend's playlist

#features scaling 
df_scaled = scaler.transform(df[features])

#use trained PCA
df_pca = pd.DataFrame(data=pca.transform(df_scaled), columns=['PC1','PC2','PC3'])

In [ ]:
df_pca = df_pca.merge(df, left_index=True, right_index=True)
df_pca = df_pca[['PC1', 'PC2', 'PC3', 'title', 'main_artist']]

df_pca.head()

### Get recomendations with PCA and Nearest Neighbors

In [ ]:
from scipy.spatial import KDTree
columns = ['PC1', 'PC2', 'PC3']

kdB = KDTree(df_pca_all_songs[columns].values)#all songs
neighbours = kdB.query(df_pca[columns].values, k=1)[-1]

print(neighbours)

#k desired number of neighbors: this case I chose 1 neighbour song
#returns indices of 1-neighbors for all rows/points in our songs "database"

In [ ]:
recomendations = all_songs[all_songs.index.isin(neighbours)]
recomendations

In [ ]:
## recomendations output 

recomendations_output = recomendations[['title', 'main_artist']]
recomendations_output

## Exploratory Data Analysis

In [ ]:
obamas = pd.read_csv('../data/obamas.csv')
pitchfork = pd.read_csv('../data/pitchfork.csv')
billboard = pd.read_csv('../data/billboard.csv')

In [ ]:
obamas['Playlist'] = 'The Obamas'
pitchfork['Playlist'] = 'Pitchfork'
billboard['Playlist'] = 'Billboard Top 100'
user_df['Playlist'] = 'Your Songs'

In [ ]:
frames = [billboard, pitchfork, obamas, user_df]

data = pd.concat(frames)
data = data.reset_index()
data = data.drop(columns=['index'])
data.head()

In [ ]:
#scale
from sklearn.preprocessing import MinMaxScaler
# Min-max scaling

data_scaled = pd.DataFrame(MinMaxScaler().fit_transform(data[features]), 
                         columns=data[features].columns)
data_scaled['Playlist'] = data['Playlist']

df_radar = data_scaled.groupby('Playlist').mean().reset_index() \
                    .melt(id_vars='Playlist', var_name="features", value_name="avg") \
                    .sort_values(by=['Playlist','features']).reset_index(drop=True)

px.line_polar(df_radar, 
              r="avg", 
              theta="features", 
              title='Mean Values of Each Track Features',
              color="Playlist", 
              line_close=True,
              line_shape='spline',
              range_r=[0, 0.9],
              color_discrete_sequence=px.colors.cyclical.mygbm[:-6])

In [ ]:
playlists = list(data['Playlist'].unique())
playlists

In [ ]:
print(features)

## Hipster or Mainstream?

Let's compare your songs to Pitchfork best albums and best new music, as well as to Billboard week's Top 100 hits

In [ ]:
Image(filename='../images/new_highfid.png',width = 500)

In [ ]:
for feature in features:
    plt.figure(figsize=(7,4))

    sns.kdeplot(data[data['Playlist']== 'Your Songs'][feature], label='Your Songs')
    sns.kdeplot(data[data['Playlist']== 'Pitchfork'][feature],   label='Pitchfork')
    sns.kdeplot(data[data['Playlist']== 'Billboard Top 100'][feature],   label='Billboard Top 100')  
    plt.grid(b=None)
    plt.title(feature)
    plt.legend()
    plt.show()


In [ ]:
Image(filename='../images/old_highfid.png',width = 300)

## You a Obama?

Let's see if you got the musical taste of a Obama

In [ ]:
for feature in features:
    plt.figure(figsize=(7,4))

    sns.kdeplot(data[data['Playlist']== 'Your Songs'][feature], label='Your Songs')
    sns.kdeplot(data[data['Playlist']== 'The Obamas'][feature],  label='The Obamas')
    plt.grid(b=None)
    plt.title(feature)
    plt.legend()
    plt.show()


In [ ]:
Image(filename='../images/obamas.png', width = 500)